## Import Python libraries

In [57]:
!pip install flwr

In [58]:
import tensorflow.compat.v1 as tf
sess = tf.Session()
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os
print(os.listdir("../input"))
import errno
import tensorflow as tf
import csv
from keras import backend as K
import sklearn
from tensorflow import keras
from keras import Sequential
from keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.layers import Conv1D, MaxPool1D
from keras.optimizers import Adam
import random
import pandas as pd
import numpy as np
import math
import os
import flwr as fl
import array
from scipy.stats import dirichlet
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

from imblearn.over_sampling import SMOTE

['adult-census-income']


In [59]:
!pip install aif360 

In [60]:
!pip install BlackBoxAuditing

## Import dataset

In [61]:
path = ('/kaggle/input/adult-census-income/adult.csv')
df = pd.read_csv(path, encoding='latin-1')

In [62]:
df.race.unique()

array(['White', 'Black', 'Asian-Pac-Islander', 'Other',
       'Amer-Indian-Eskimo'], dtype=object)

## Exploratory Data Analysis

### Check shape of dataset

In [63]:
df.shape

(32561, 15)

We can see that there are 32561 instances and 15 attributes in the data set.

### Preview dataset

In [64]:
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [65]:
df.race.value_counts()

White                 27816
Black                  3124
Asian-Pac-Islander     1039
Amer-Indian-Eskimo      311
Other                   271
Name: race, dtype: int64

### View summary of dataframe

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


Summary of the dataset shows that there are no missing values. But the preview shows that the dataset contains values coded as `?`. So, I will encode `?` as NaN values.

### Encode `?` as `NaNs`

In [67]:
df[df == '?'] = np.nan

In [68]:
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,NaN,77053,HS-grad,9,Widowed,NaN,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,NaN,186061,Some-college,10,Widowed,NaN,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [69]:
df.isna().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     583
income               0
dtype: int64

### Impute missing values with mode

In [70]:
for col in ['workclass', 'occupation', 'native.country']:
    df[col].fillna(df[col].mode()[0], inplace=True)

### Check again for missing values

In [71]:
df.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
income            0
dtype: int64

In [72]:
import numpy as np
import pandas as pd

def race_Dirichelet_sampling(data, alphas, values, n_lines):
    assert (n_lines <= len(data))
    assert (len(values) == len(alphas))

    s = np.random.dirichlet(tuple(alphas), 1).tolist()[0]
    groups = []

    for i in range(len(values)):
        if round(n_lines * s[i]) > 0:
            groups.append(data[data['race'] == values[i]].sample(n=round(n_lines * s[i]), replace=True))
        else:
            groups.append(data[data['race'] == values[i]].sample(n=1))

    return pd.concat(groups)
alphas = [1, 2, 3, 1, 1]  
values = ['White', 'Black', 'Asian-Pac-Islander', 'Other', 'Amer-Indian-Eskimo']
n_lines = 1000 
result_dataset = race_Dirichelet_sampling(df, alphas, values, n_lines)


In [73]:
alphas = [0.5, 0.2, 0.1, 0.1, 0.2]  
values = ['White', 'Black', 'Asian-Pac-Islander', 'Other', 'Amer-Indian-Eskimo']
n_lines = 1000 
client1 = race_Dirichelet_sampling(df, alphas, values, n_lines)


In [74]:
values = ['White', 'Black', 'Asian-Pac-Islander', 'Other',
       'Amer-Indian-Eskimo']
df1 = race_Dirichelet_sampling(df, [0.5, 0.2, 0.1, 0.1, 0.2] , values, 11000)
df2 = race_Dirichelet_sampling(df, [0.2,0.5,0.7,0.6,0.9], values, 10000)
df3 = race_Dirichelet_sampling(df, [0.1,0.6,0.8,0.4,0.7], values, 10000)

In [75]:
client1.race.value_counts()

Other                 936
Amer-Indian-Eskimo     46
White                  14
Black                   3
Asian-Pac-Islander      1
Name: race, dtype: int64

In [76]:
df.race.value_counts()

White                 27816
Black                  3124
Asian-Pac-Islander     1039
Amer-Indian-Eskimo      311
Other                   271
Name: race, dtype: int64

In [77]:

client1.race.value_counts()

Other                 936
Amer-Indian-Eskimo     46
White                  14
Black                   3
Asian-Pac-Islander      1
Name: race, dtype: int64

Now we can see that there are no missing values in the dataset.

### Setting feature vector and target variable

In [78]:
X = df.drop(['income'], axis=1)
y = df['income']

In [79]:
y.unique(),y.nunique()

(array(['<=50K', '>50K'], dtype=object), 2)

In [80]:
X.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,90,Private,77053,HS-grad,9,Widowed,Prof-specialty,Not-in-family,White,Female,0,4356,40,United-States
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States
2,66,Private,186061,Some-college,10,Widowed,Prof-specialty,Unmarried,Black,Female,0,4356,40,United-States
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States


## Split data into separate training and test set

In [81]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

## Feature Engineering

### Encode categorical variables: LabelEncoder

![](https://ai-ml-analytics.com/wp-content/uploads/2020/08/encoding-3.png)

In [82]:
from sklearn import preprocessing

categorical = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']
for feature in categorical:
        le = preprocessing.LabelEncoder()
        X_train[feature] = le.fit_transform(X_train[feature])
        X_test[feature] = le.transform(X_test[feature])

In [83]:
X = pd.concat([X_train, X_test], axis=0)
y = pd.concat([y_train, y_test], axis=0)
dff = pd.concat([X, y], axis=1)
dff=dff.reset_index(drop=True)

In [84]:
df.race.unique()

array(['White', 'Black', 'Asian-Pac-Islander', 'Other',
       'Amer-Indian-Eskimo'], dtype=object)

In [85]:
df1.shape,df2.shape, df3.shape

((11002, 15), (10000, 15), (10001, 15))

In [86]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

## Feature Scaling

![](https://cdn-images-1.medium.com/max/370/1*Nlgc_wq2b-VfdawWX9MLWA.png)

In [87]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)

X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)

In [88]:
X_train.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,0.101484,2.600478,-1.494279,-0.332263,1.133894,-0.402341,-0.782234,2.214196,0.39298,-1.430470,-0.145189,-0.217407,-1.662414,0.262317
1,0.028248,-1.884720,0.438778,0.184396,-0.423425,-0.402341,-0.026696,-0.899410,0.39298,0.699071,-0.145189,-0.217407,-0.200753,0.262317
2,0.247956,-0.090641,0.045292,1.217715,-0.034095,0.926666,-0.782234,-0.276689,0.39298,-1.430470,-0.145189,-0.217407,-0.038346,0.262317
3,-0.850587,-1.884720,0.793152,0.184396,-0.423425,0.926666,-0.530388,0.968753,0.39298,0.699071,-0.145189,-0.217407,-0.038346,0.262317
4,-0.044989,-2.781760,-0.853275,0.442726,1.523223,-0.402341,-0.782234,-0.899410,0.39298,0.699071,-0.145189,-0.217407,-0.038346,0.262317


****

# adversarial debiasing :

In [89]:
!pip install aif360

from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import AdversarialDebiasing

In [90]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score

y_train_df = pd.DataFrame({'income': y_train})

dataset_train = StandardDataset(
    df=X_train.join(y_train_df),
    label_name='income',
    favorable_classes=[1],
    protected_attribute_names=['race', 'sex'],
    privileged_classes=[['1'], ['1']]
)
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
debias_model = AdversarialDebiasing(
    unprivileged_groups=[{'race': 0, 'sex': 0}],
    privileged_groups=[{'race': 1, 'sex': 1}],
    scope_name='adversarial_debiasing',
    sess=sess
)
debias_model.fit(dataset_train)
y_test_df = pd.DataFrame({'income': y_test})
dataset_test = StandardDataset(
    df=X_test.join(y_test_df),
    label_name='income',
    favorable_classes=[1],
    protected_attribute_names=['race', 'sex'],
    privileged_classes=[['1'], ['1']]
)
dataset_debiased = debias_model.predict(dataset_test)
# Evaluate the performance of the debiased model
y_pred = dataset_debiased.labels
y_true = dataset_test.labels.ravel()


epoch 0; iter: 0; batch classifier loss: 0.748442; batch adversarial loss: 0.775368
epoch 1; iter: 0; batch classifier loss: 0.453324; batch adversarial loss: 0.627236
epoch 2; iter: 0; batch classifier loss: 0.244760; batch adversarial loss: 0.535942
epoch 3; iter: 0; batch classifier loss: 0.311097; batch adversarial loss: 0.418486
epoch 4; iter: 0; batch classifier loss: 0.298092; batch adversarial loss: 0.422298
epoch 5; iter: 0; batch classifier loss: 0.251771; batch adversarial loss: 0.379531
epoch 6; iter: 0; batch classifier loss: 0.262738; batch adversarial loss: 0.413025
epoch 7; iter: 0; batch classifier loss: 0.295021; batch adversarial loss: 0.319276
epoch 8; iter: 0; batch classifier loss: 0.323505; batch adversarial loss: 0.167808
epoch 9; iter: 0; batch classifier loss: 0.314794; batch adversarial loss: 0.391776
epoch 10; iter: 0; batch classifier loss: 0.285047; batch adversarial loss: 0.202220
epoch 11; iter: 0; batch classifier loss: 0.357243; batch adversarial loss:

In [91]:
print("Accuracy score:", accuracy_score(y_true, y_pred))
print("Balanced accuracy score:", balanced_accuracy_score(y_true, y_pred))
print("F1 score:", f1_score(y_true, y_pred))

Accuracy score: 0.8450199611014434
Balanced accuracy score: 0.7355921474537748
F1 score: 0.6201705970898143
